In [ ]:
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
%matplotlib inline 
%load_ext memory_profiler
from sklearn.metrics import make_scorer
from scipy.special import expit
import time
import math
import random
from memory_profiler import memory_usage
from sklearn import metrics as mt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc
from scipy import interp

target_classifier = 'PC'
df = pd.read_csv('responses.csv', sep=",")

In [ ]:
# remove rows whose target classfier value is NaN
df_cleaned_classifier = df[np.isfinite(df[target_classifier])]
# change NaN number values to the mean
df_imputed = df_cleaned_classifier.fillna(df.mean())
# get categorical features
object_features = list(df_cleaned_classifier.select_dtypes(include=['object']).columns)

df_imputed = df_imputed.dropna()

### Removing Features

In [ ]:
df = df_imputed[
    ["Music","Dance","Folk","Country","Classical music","Musical",
    "Pop","Rock","Metal or Hardrock","Punk","Hiphop, Rap","Reggae, Ska",
    "Swing, Jazz","Rock n roll","Alternative","Latino","Techno, Trance",
    "Opera","Movies","Horror","Thriller","Comedy","Romantic","Sci-fi","War",
    "Fantasy/Fairy tales","Animated","Documentary","Western","Action","History",
    "Psychology","Politics","Mathematics","Physics","Internet","PC","Economy Management",
    "Biology","Chemistry","Reading","Geography","Foreign languages","Medicine","Law",
    "Cars","Art exhibitions","Religion","Countryside, outdoors","Dancing",
    "Musical instruments","Writing","Passive sport","Active sport","Gardening","Celebrities",
    "Shopping","Science and technology","Theatre","Fun with friends","Adrenaline sports",
    "Pets","Healthy eating","Spending on gadgets","Age","Height","Weight",
    "Number of siblings","Gender","Left - right handed","Education","Village - town","House - block of flats"
]]

We removed phobias and other financial information from the poll. Both sources of data are not readily available. We also removed 'Only Child' because it is redundant with number of siblings.

### Label Features

### Create Cross-Product Features

In [12]:
Cross_product_features = [['Age', 'Gender', 'Folk', 'Country', 'Western movies'], ['Age', 'Gender', 'Classical', 'Swing, Jazz', 'Opera'], ['Age', 'Gender','Musicals', 'Pop'], ['Age', 'Gender', 'Rock', 'Metal or Hardrock', 'Rock n roll', 'Hiphop, Rap', 'Reggae, Ska'], ['Age', 'Gender', 'Alternative'], ['Age', 'Gender', 'Latin'], ['Age', 'Gender', 'Techno, Trance'], ['Horror', 'Thriller'], ['Comedy', 'Romantic'], ['Sci-fi', 'Fantasy/Fairy tails', 'Animated'], ['War', 'Action'], ['History', 'Geography', 'Foreign Languages'], ['Psychology', 'Biology', 'Medicine'], ['Mathematics', 'Physics', 'Science and technology'], ['Economy Management', 'Politics', 'Law', 'Voting'], ['Biology', 'Physics', 'Chemistry'], ['Reading', 'Writing'], ['Art exhibitions', 'Theatre', 'Musical instruments'], ['Countryside, outdoors', 'Dancing', 'Active sport', 'Adrenaline sports'], ['Passive sport', 'Gardening'], ['Dancing', 'Musical instruments'], ['Smoking', 'Alcohol'], ['Age','Height', 'Weight'], ['Village-town', 'House-block of flats']]
# create from 2d array above

We put features into groups that we thought would be clustered (feature space closer together). That way when the features are not clustered it gives us more information to predict.

In [12]:
df_fixed['Spending on gadgets'][1:]

1       5
2       4
3       4
4       2
5       4
6       1
7       3
8       3
9       2
10      2
11      1
12      2
13      3
14      1
15      5
16      3
17      2
18      5
19      3
20      4
21      2
22      4
23      2
24      5
25      1
26      4
27      2
28      1
29      2
30      4
       ..
980     4
981     2
982     3
983     3
984     4
985     3
986     1
987     4
988     5
989     1
990     4
991     3
992     1
993     4
994     3
995     2
996     3
997     2
998     2
999     3
1000    4
1001    2
1002    3
1003    4
1004    4
1005    3
1006    5
1007    2
1008    3
1009    1
Name: Spending on gadgets, dtype: int64

In [4]:
# Research on Cost Matrix
# http://www.ibm.com/support/knowledgecenter/SSEPGG_11.1.0/com.ibm.im.model.doc/c_cost_matrix.html

cost_matrix = np.matrix([[0,1,2,3,4],
[1,0,1,2,3],
[3,1,0,1,2],
[5,3,1,0,1],
[7,5,2,1,0]])

def get_confusion_costTot(confusion_matrix, cost_matrix):
    score = np.sum(confusion_matrix*cost_matrix)
    return score

confusion_scorer = make_scorer(get_confusion_costTot, greater_is_better=False)
confusion_scorer

make_scorer(get_confusion_costTot, greater_is_better=False)

## Creating Cross-Product Features

## Divide Data into Training and Testing

## Creating Tensorflow Network

In [6]:
for train_indices, test_indices in cv_object.split(X,y): 
        # I will create new variables here so that it is more obvious what 
        # the code is doing (you can compact this syntax and avoid duplicating memory,
        # but it makes this code less readable)
        X_train = (X[train_indices])
        y_train = y[train_indices]

    #     print(X_train)
    #     print(y_train)

        X_test = (X[test_indices])
        y_test = y[test_indices]

In [14]:
from copy import deepcopy
# df_train = deepcopy(X_train)
# df_test = deepcopy(y_test)
# df_train = X_train
# df_test = y_test
df_train = df
df_test = df

In [15]:
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.contrib import layers
from tensorflow.contrib.learn.python import SKCompat
tf.logging.set_verbosity(tf.logging.WARN) # control the verbosity of tensor flow

In [16]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# if df_test.income.dtype=='object':
#     df_test.income.replace(to_replace=[' <=50K.',' >50K.'],value=['<=50K','>50K'],inplace=True)
#     print(df_test.income.value_counts())

encoders = dict() 
categorical_headers = ['Smoking habits']

for col in categorical_headers+['Spending on gadgets']:
    print(col)
#     df_train[col] = df_train[col].str.strip()
#     df_test[col] = df_test[col].str.strip()
    if col=="Spending on gadgets":
        tmp = LabelEncoder()
        df_train[col] = tmp.fit_transform(df_train[col])
        df_test[col] = tmp.transform(df_test[col])
    else:
        print("HI")
#         encoders[col] = LabelEncoder()
#         df_train[col] = encoders[col].fit_transform(df_train[col])
#         df_test[col] = encoders[col].transform(df_test[col])
#         df_train[col+'_int'] = encoders[col].fit_transform(df_train[col])
#         df_test[col+'_int'] = encoders[col].transform(df_test[col])


numeric_headers = ['Internet','Smoking','Math']

for col in numeric_headers:
    df_train[col] = df_train[col].astype(tf.float)
    df_test[col] = df_test[col].astype(tf.float)
    
    ss = StandardScaler()
    df_train[col] = ss.fit_transform(df_train[col].values.reshape(-1, 1))
    df_test[col] = ss.transform(df_test[col].values.reshape(-1, 1))

Smoking habits
HI
Spending on gadgets


ValueError: y contains new labels: [0]

In [10]:
df_train["Spending on gadgets"]

/Users/rupalsanghavi/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if __name__ == '__main__':


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [17]:
df

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,Age,Height,Weight,Number of siblings,Gender,Left - right handed,Education,Only child,Village - town,House - block of flats
0,5.0,3.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,...,20.0,163.0,48.0,1.0,female,right handed,college/bachelor degree,no,village,block of flats
1,4.0,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0,...,19.0,163.0,58.0,2.0,female,right handed,college/bachelor degree,no,city,block of flats
2,5.0,5.0,2.0,2.0,3.0,4.0,5.0,3.0,5.0,3.0,...,20.0,176.0,67.0,2.0,female,right handed,secondary school,no,city,block of flats
3,5.0,3.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,...,22.0,172.0,59.0,1.0,female,right handed,college/bachelor degree,yes,city,house/bungalow
4,5.0,3.0,4.0,3.0,2.0,4.0,3.0,5.0,3.0,1.0,...,20.0,170.0,59.0,1.0,female,right handed,secondary school,no,village,house/bungalow
5,5.0,3.0,2.0,3.0,2.0,3.0,3.0,2.0,5.0,5.0,...,20.0,186.0,77.0,1.0,male,right handed,secondary school,no,city,block of flats
6,5.0,5.0,5.0,3.0,1.0,2.0,2.0,5.0,3.0,1.0,...,20.0,177.0,50.0,1.0,female,right handed,secondary school,no,village,house/bungalow
7,5.0,3.0,3.0,2.0,1.0,2.0,2.0,4.0,5.0,1.0,...,19.0,184.0,90.0,1.0,male,right handed,college/bachelor degree,no,city,house/bungalow
8,5.0,3.0,3.0,1.0,1.0,2.0,4.0,3.0,5.0,5.0,...,18.0,166.0,55.0,1.0,female,right handed,secondary school,no,city,house/bungalow
9,5.0,3.0,2.0,5.0,2.0,2.0,5.0,3.0,5.0,2.0,...,19.0,174.0,60.0,3.0,female,right handed,secondary school,no,city,block of flats


In [5]:
from sklearn.model_selection import ShuffleSplit

# we want to predict the X and y data as follows:
if target_classifier in df_fixed:
    y = df_fixed[target_classifier].values # get the labels we want
    del df_fixed[target_classifier] # get rid of the class label
    X = df_fixed.values # use everything else to predict!

X = X/5
num_folds = 10

cv_object = StratifiedKFold(n_splits= num_folds, random_state=None, shuffle=True)
cv_object.split(X,y)

print(cv_object)

StratifiedKFold(n_splits=10, random_state=None, shuffle=True)
